In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(168)
cluster

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://172.30.100.3:42073 Dashboard: http://172.30.100.3:46751/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7610482     hsw24 make_pro albert7a PD       0:00      1 (None)
           7610483     hsw24 make_pro albert7a PD       0:00      1 (None)
           7610484     hsw24 make_pro albert7a PD       0:00      1 (None)
           7610485     hsw24 make_pro albert7a PD       0:00      1 (None)
           7610486     hsw24 make_pro albert7a PD       0:00      1 (None)
           7610487     hsw24 make_pro albert7a PD       0:00      1 (None)
           7610461     hsw24 make_pro albert7a  R       1:19      1 n2398
           7610457     hsw24 make_pro albert7a  R       4:12      1 n2764
           7610458     hsw24 make_pro albert7a  R       4:12      1 n2373
           7610459     hsw24 make_pro albert7a  R       4:12      1 n2375
           7610460     hsw24 make_pro albert7a  R       4:12      1 n2376
           7610419     hsw24 make_pro albert7a  R      16:40      1 n2268
           7610420   

In [4]:
import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

10


In [5]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7610483     hsw24 make_pro albert7a PD       0:00      1 (Resources)
           7610484     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           7610485     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           7610486     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           7610487     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           7610482     hsw24 make_pro albert7a  R       0:18      1 n2185
           7610461     hsw24 make_pro albert7a  R       1:41      1 n2398
           7610457     hsw24 make_pro albert7a  R       4:34      1 n2764
           7610458     hsw24 make_pro albert7a  R       4:34      1 n2373
           7610459     hsw24 make_pro albert7a  R       4:34      1 n2375
           7610460     hsw24 make_pro albert7a  R       4:34      1 n2376
           7610419     hsw24 make_pro albert7a  R      17:02      1 n2268
 

In [6]:
import sys
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import cmocean
import glob

%matplotlib inline


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:16: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [7]:
#params

plot_eke_monthly=False
plot_eke_12_month=True
plot_eke_annual=False


In [8]:
config='NATL60'
case='CJM165'
freq='1d'
#year='2010' #depends on the month

datadir='/store/molines/'+str(config)+'/'+str(config)+'-'+str(case)+'-S/'+str(freq)+'/'
#!ls $datadir
gridfile='/store/molines/NATL60/NATL60-I/NATL60_coordinates_v4.nc'
maskfile='/store/molines/NATL60/NATL60-I/NATL60_v4.1_cdf_byte_mask.nc'

In [9]:
dsgrid=xr.open_dataset(gridfile)
dsmask=xr.open_dataset(maskfile)

In [10]:
def compute_EKE(u,v,u2mean,v2mean):
    ua=0.5*((u2mean-u*u)+(u2mean.shift(x=-1)-u.shift(x=-1)*u.shift(x=-1)))
    va=0.5*((v2mean-v*v)+(v2mean.shift(y=-1)-v.shift(y=-1)*v.shift(y=-1)))
    uat=ua.rename({'depthu':'deptht'})
    vat=va.rename({'depthv':'deptht'})
    eke=0.5*(uat+vat)
    return eke


In [11]:
month_name={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
def plot_monthly(var,mask,vmin,vmax,cmap,title,figfile):
    fig, axes = plt.subplots(3,4,figsize=(30,20))
    axes = axes.ravel()
    for m in np.arange(1,13):
        if m < 10:
            month='0'+str(m)
        else:
            month=str(m)
        if m in [1,3,5,7,8,10,12]:
            dayf='31'
        elif m == 2:
            dayf='28'
        else:
            dayf='30'
        if m < 10:
            year='2013'
        else:
            year='2012'
        ax=axes[m-1]
        var.sel(time_counter=slice(str(year)+'-'+month+'-01',str(year)+'-'+month+'-'+dayf)).mean(dim='time_counter').where(mask==1).plot(cmap=cmap,ax=ax,vmin=vmin,vmax=vmax,add_labels=True)
        ax.set_title(month_name[m])
    fig.suptitle('Monthly mean of '+str(title))
    plt.savefig(config+'-'+case+'y'+str(year)+'.1m_'+figfile+'.png')

In [12]:
month_name={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
def plot_one_month(var,mask,vmin,vmax,cmap,title,figfile,m):
    fig, ax = plt.subplots(1,1,figsize=(10,15))
    if m < 10:
        month='0'+str(m)
    else:
        month=str(m)
    if m in [1,3,5,7,8,10,12]:
        dayf='31'
    elif m == 2:
        dayf='28'
    else:
        dayf='30'
        if m < 10:
            year='2013'
        else:
            year='2012'
    var.sel(time_counter=slice(str(year)+'-'+month+'-01',str(year)+'-'+month+'-'+dayf)).mean(dim='time_counter').where(mask==1).plot(cmap=cmap,ax=ax,vmin=vmin,vmax=vmax,add_labels=True)
    ax.set_title(month_name[m])
    fig.suptitle('Monthly mean of '+str(title))
    plt.savefig(config+'-'+case+'y'+str(year)+'m'+str(month)+'_'+figfile+'.png')

In [13]:
def plot_annual(var,mask,vmin,vmax,cmap,title,figfile):
    fig, ax = plt.subplots(1,1,figsize=(10,15))
    var.mean(dim='time_counter').where(mask==1).plot(cmap=cmap,ax=ax,vmin=vmin,vmax=vmax,add_labels=True)
    fig.suptitle('Annual mean of '+str(title))
    plt.savefig(config+'-'+case+'_y'+str(year)+'.1y_'+figfile+'.png')    

In [14]:
if plot_eke_monthly==True:
    filesU=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridU.nc'))
    filesV=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridV.nc'))
    dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':1,'depthu':1})
    dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':1,'depthv':1})
    u=dsU.vozocrtx[:,:,424:3286,3858:5076]
    v=dsV.vomecrty[:,:,424:3286,3858:5076]
    mask=dsmask.tmask[0,:,424:3286,3858:5076]
    u2=u*u
    v2=v*v
    u2mean=u2.mean(dim='time_counter')
    v2mean=v2.mean(dim='time_counter')
    eke=compute_EKE(u,v,u2mean,v2mean)
    plot_monthly(10000*eke[0],mask[0],0,2500,'cmo.amp','Surface EKE','eke_surf')


In [ ]:
if plot_eke_12_month==True:
    filesU=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridU.nc'))
    filesV=sorted(glob.glob(datadir+'/*/'+str(config)+'-'+str(case)+'_y????m??d??.'+str(freq)+'_gridV.nc'))
    dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':1,'depthu':1})
    dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':1,'depthv':1})
    u=dsU.vozocrtx[:,:,424:3286,3858:5076]
    v=dsV.vomecrty[:,:,424:3286,3858:5076]
    mask=dsmask.tmask[0,:,424:3286,3858:5076]
    u2=u*u
    v2=v*v
    u2mean=u2.mean(dim='time_counter')
    v2mean=v2.mean(dim='time_counter')
    eke=compute_EKE(u,v,u2mean,v2mean)
    for m in np.arange(8,13):
        plot_one_month(10000*eke[0],mask[0],0,2500,'cmo.amp','Surface EKE','eke_surf',m)
   